In [64]:
import pandas as pd
import tensorflowjs as tfjs
import keras
import numpy as np
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Dropout, Flatten
from keras.utils import to_categorical
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

In [65]:
keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

In [237]:
df = pd.read_csv("../data/training-candles.csv")

In [238]:
df.head()

,mts,open,close,high,low,volume,pair,timeframe,upDownThreeMethods,upsideGapTwoCrows,...,minus_dm,plus_di,minus_di,cvi,adx,apo,rocr,percentChange,strategy,strategyCode
0,1526508900000,0.68934,0.68807,0.68964,0.68786,92037.635296,tXRPUSD,15m,0,0,...,0.158281,0.541802,0.130054,0.284717,0.305497,0.626824,1.007659,-0.332815,LONG,1
1,1526509800000,0.68807,0.69337,0.69654,0.68798,342806.832130,tXRPUSD,15m,0,0,...,0.139982,0.691945,0.096433,0.415318,0.363192,0.679183,1.014871,0.764383,SHORT,0
2,1526510700000,0.69337,0.69205,0.69345,0.69042,79685.963285,tXRPUSD,15m,0,0,...,0.122990,0.654195,0.085807,0.453708,0.416766,0.710954,1.016226,-0.190738,LONG,1
3,1526511600000,0.69206,0.69478,0.69667,0.69150,262595.672256,tXRPUSD,15m,0,0,...,0.107212,0.704697,0.068685,0.601596,0.474464,0.751934,1.023919,0.392930,SHORT,0
4,1526512500000,0.69440,0.69451,0.69656,0.69303,169990.366154,tXRPUSD,15m,0,0,...,0.092561,0.659651,0.057741,0.621157,0.528040,0.780616,1.022978,-0.038876,LONG,1


In [239]:
def create_model(optimizer="adam", init_mode="uniform", activation="sigmoid"):
    model = Sequential()

    model.add(Dense(48, kernel_initializer=init_mode, input_shape=(46,), activation=activation))
    model.add(Dropout(0.2))
    model.add(Dense(24, kernel_initializer=init_mode, activation=activation))
    model.add(Dropout(0.2))
    model.add(Dense(12, kernel_initializer=init_mode, activation=activation))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer=init_mode, activation="sigmoid"))

    model.compile(loss="binary_crossentropy",
                  optimizer=optimizer,
                  metrics=["accuracy"])
    return model

In [240]:
X = df[["adx", "cci", "rocr", "hikkake", "harami", "closingMarubozu", 
        "onNeck", "longLineCandle", "hikkakeModified", "apo", "cmo", "adx",
        "cvi", "plus_dm", "minus_dm", "plus_di", "minus_di", "dpo", "kvo",
        "fosc", "fisher", "fisher_signal", "dx", "linregslope", "macd", "macd_signal", 
        "macd_histogram", "mfi", "mom", "obv", "ppo", "pvi", "rsi", "stoch_k", "stoch_d",
        "atr", "trix", "ultosc", "vosc", "willr", "cmo", "cci", "adosc", "adxr", "ao",
        "aroonosc"
       ]].values

y = df["strategyCode"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [209]:
model = KerasClassifier(build_fn=create_model, verbose=0)

batch_size = [128]
epochs = [100]
optimizer = ['Adadelta', 'Adam']
init_mode = ['normal', 'uniform']
activation = ['relu', 'softplus', 'sigmoid']

param_grid = dict(batch_size=batch_size, epochs=epochs, optimizer=optimizer, init_mode=init_mode, activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

In [210]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.556526 using {'activation': 'softplus', 'batch_size': 128, 'epochs': 100, 'init_mode': 'normal', 'optimizer': 'Adam'}
0.537538 (0.006668) with: {'activation': 'relu', 'batch_size': 128, 'epochs': 100, 'init_mode': 'normal', 'optimizer': 'Adadelta'}
0.537320 (0.011068) with: {'activation': 'relu', 'batch_size': 128, 'epochs': 100, 'init_mode': 'normal', 'optimizer': 'Adam'}
0.539284 (0.002561) with: {'activation': 'relu', 'batch_size': 128, 'epochs': 100, 'init_mode': 'uniform', 'optimizer': 'Adadelta'}
0.551506 (0.005115) with: {'activation': 'relu', 'batch_size': 128, 'epochs': 100, 'init_mode': 'uniform', 'optimizer': 'Adam'}
0.542994 (0.011447) with: {'activation': 'softplus', 'batch_size': 128, 'epochs': 100, 'init_mode': 'normal', 'optimizer': 'Adadelta'}
0.556526 (0.014336) with: {'activation': 'softplus', 'batch_size': 128, 'epochs': 100, 'init_mode': 'normal', 'optimizer': 'Adam'}
0.545832 (0.004886) with: {'activation': 'softplus', 'batch_size': 128, 'epochs': 100, 'in

In [241]:
optimal_model = create_model(optimizer="Adam", init_mode="normal", activation="softplus")

#y_labels = to_categorical(y_train, num_classes=2)

optimal_model.fit(X_train, y_train, epochs=100, batch_size=128, verbose=1)

Epoch 1/100
4582/4582 [==============================] - 9s 2ms/step - loss: 0.6936 - acc: 0.4980
Epoch 2/100
4582/4582 [==============================] - 0s 34us/step - loss: 0.6927 - acc: 0.5225
Epoch 3/100
4582/4582 [==============================] - 0s 35us/step - loss: 0.6928 - acc: 0.5120
Epoch 4/100
4582/4582 [==============================] - 0s 33us/step - loss: 0.6922 - acc: 0.5131
Epoch 5/100
4582/4582 [==============================] - 0s 32us/step - loss: 0.6925 - acc: 0.5085
Epoch 6/100
4582/4582 [==============================] - 0s 32us/step - loss: 0.6932 - acc: 0.5146
Epoch 7/100
4582/4582 [==============================] - 0s 32us/step - loss: 0.6918 - acc: 0.5214
Epoch 8/100
4582/4582 [==============================] - 0s 37us/step - loss: 0.6916 - acc: 0.5249
Epoch 9/100
4582/4582 [==============================] - 0s 33us/step - loss: 0.6911 - acc: 0.5323
Epoch 10/100
4582/4582 [==============================] - 0s 32us/step - loss: 0.6904 - acc: 0.5310
Epoch 11/1

In [242]:
#y_test_labels = to_categorical(y_test, num_classes=2)
evaluation = optimal_model.evaluate(x=X_test, y=y_test, steps=1000)
optimal_model.metrics_names
print(evaluation)

1000/1000 [==============================] - 7s 7ms/step
[0.7050487995147705, 0.5287865400314331]


In [243]:
tfjs.converters.save_keras_model(optimal_model, "../public/model/")

In [214]:
optimal_model.predict_classes(X_test)

array([[0],
       [0],
       [0],
       ...,
       [1],
       [0],
       [0]], dtype=int32)